# Escalabilidad en el backend

## ¿Qué es el escalamiento?

### Escalamiento vertical.

![](https://i.imgur.com/SNnfqIP.png)

Es cuando tenemos un servidor y eventualmente el servidor empieza a colapsar, una forma de solucionar el problema es incrementar:

- CPU
- RAM
- Disk

Problemas:

*Costos*

En un black friday tu puedes escalar la aplicacion, pero algunos provedores no permiten desescalar.

### Escalamiento horizontal

Soluciona el problema de la disponibilidad. Se tienen varias instancias del mismo servidor.
Como se tienen distintos servidores ahora se necesita de un:

LOAD BALANCER - Balcnaceador de carga

![](https://i.imgur.com/D3u2vZf.png)

LOAD BALANCER: Tiene conocimiento de nuestras instancias/servidores (al conjunto de servidores se denomina Clouster).

Si un nodo(instancia) se cae, el load balancer se encarga de desviarla. Distribuye las peticiones.

El Escalamiento Horizontal es mas complejo de configurar. 

#### Problema

Si tenemos la base de datos local en cada servidor va haber un problema dado que no se tienen sincronizados los datos de las distintas bases de datos de los servidores. **SOLUCION** Gestionar la base de datos fuera de estos servidores(BDaS). Quizas como un servidor aparte que sirva como DB.

Si estas gestionando la BD como servicio con cualquiera de los proveedores mencionados, no tendras que preocuparte. Sin embargo si la estas gestionando dentro de esos servidores. Y tendrias que hacer algo que se llama **replicacaion**



## ¿Qué es la replicación?

Partiendo del escalamiento horizontal, tal y como lo vimos anteriormente los datos no estaban sincronizados entre si. Y entonces?

Una solucion podria ser aislar la BD en un servidor en particular. Pero hay un probelma, a pesar de tener replicacion horizontal en los servicios de backend, volvemos al mismo punto, que es la BD. Y una forma sencilla de solucionarlo seria hacer escalamiento vertical solo sobre el servidsor que contiene la BD.

![](https://i.imgur.com/zTqttH5.png)


Pero a medida que aumenta la complejidad del sistema, podriamos nuevamente aplicar el escalameinto horizontal en la BD en pèrse. E instalar un Load Balancer para que maneje las peticiones desde el *backend* hacia la BD. 

![](https://i.imgur.com/X6SgQkw.png)

Y volvemos al problema de la seccion anterior. COmo lo vamos a solucionar?

### Sincronizacion de las BDs

Cada vez que haya por ejemplo una escritura a una cualquiera de la tres instancias de la BD, hay un proceso de sincronizacion entre las mismas, conocido como replicacion. 

![](https://i.imgur.com/UmuTRJG.png)

COmo Backend no nos encargaremos en estos asuntos, a menos que quieras ser adminstrador de bases de datos. Por lo general se delega esto a un tercero o proveedor. 

## Que es el CACHE

A procedimientos que ya se habian realizado los guardamos en memoria para que en futuras consultas haya menos latencia.

![](https://i.imgur.com/nEHdY68.png)

Si por ejemplo hay una nueva solicitud hacia el endpoint de cursos en platzi. Antes de empezar a resolver esa peticion preguntamos "esa peticion esta en cache"? 

Cada una de estas peticiones que se guardan en cache tienen un tiempo de expiracion, no pueden estar ahi para siempre. 

Esta logica estara implementada en nuestro servidor.

![](https://i.imgur.com/Nsv61G6.png)

### Donde Implementarlo

- eCommerce 
- Sitios de NOticias
- Blogs

Bien implementado el sistema de CACHE puede servir para minimizar los efectos de un ataque de DDOS

https://www.cloudflare.com/learning/ddos/memcached-ddos-attack/



## Colas de Tareas

Como y en que escenario implementarlos. Hay cierto tipo de  tareas como reportes, backups, graficos, zips, PDFs, CSVs que demandan muchos recursos 

SI hacemos una peticion de estas tares, primero ningun usuario va a aguantarse 10 minutos a que se le entregue, y segundo se estan usando recursos que podrian usarse para responder otras peticiones a otros usuarios. 

### Como evitar esto 

Consideremos que Un request se ejecuta lo más pronto posible y responde por el mismo medio. COn una cola de tareas:

"Una cola de tareas eventualmente ejecuta un proceso y puede responder por otros medios". SIgnifica que eventualmente, no lo respondera de forma instantanea, se tomara un pequeño tiempo, y segundo lo hara por otros medios.

Por ejemplo en FB, hay una opcion en la cual puedes sacar un BCKUP de toda tu informacion, si le das opcion, te respondera que tardara un tiempo mientras realiza el proceso. Y una vez que responden, lo haces es por otro medio: un mail para este caso. 

Entonces se le da una response al usuario, no con lo que pidio, sino que se va a realizar y eventualmente le llegara su correo con la peticion original. 😀

#### Encolar una tarea

COmo quine va y hace una fila en un banco: 

![](https://i.imgur.com/nwpnay9.png)

Atiendo en el orden que van llegado, y no todas a la vez. "Encolamos las tareas" dicen los ingenieros. 

#### Como entran las colas al sistema

Podriamos tener un servidor enfocado a ese tipo de tareas, y las va almacenando en el orden que van llegando. Mira el rol que cumple el load balancer en todo este proceso

![](https://i.imgur.com/D1Q8sNa.png)


## Server SIde Rendering

Partamos de que tenemos nuestros servicios de backend que recuerda, pueden retornar diferentes tipos de archivos:

![](https://i.imgur.com/4a2bema.png)

Por ejemplo, cuando hizimos la consulta la Fake Api de Platzi, esta retorno un JSON, y los Clientes consumen esta data, y lo renderizan en sus equipos o navegadores. 

Recuerda tambien podemos retornar HTML, el servidor renderiza el HTML y no espera que el cliente lo haga: *Server SIde Rendering*

![](https://i.imgur.com/l8lsWwV.png)

Y le ahorramos trabajo al navegador. QUe no es iteractivo significa, por ejemplo que en Twitter das me gusta, e inmediatamente sube el contador de los corazoncitos.

Por eso los sitios que hacen renderizado desde el servidor, no son iteractivos, precismente porque por cada render, les toca volvera a recargar toda la pagina. 

### ALgo ha cambiado

Muchas veces se prefiere se haga el renderizado desde el cliente, y ya no se consumen los datos html puro, sino JSON.

![](https://i.imgur.com/lQMeQRi.png)

Frameworks de Frontend como:

- Angular 
- VueJS
- React

Nos ayudan a realizar el renderizado del lado del cliente

### Se podria tener los mejor de los opciones.

Hay una nueva generacion que hacen renderizado desde el lado del servidor, y luego de forma dinamica, como estan pensados para FrontEnd, empiezan a ser bastante interactivos. 

![](https://i.imgur.com/zSNTAAX.png)

Como Backend developer seguramente tendras que soportar equipos de FronteEnd que quieren llevar esto al servidor, 

Podriamos tener una arquitectura como esta:

![](https://i.imgur.com/cnGW4vU.png)

EN esta arquitectura tenemos un servidor adicional dedicado al renderizado, estos servidores generalmente funionan con JS(node JS). 

Muchas veces para hacer este renderizado desde el lado del servidor se recurre tambien a proveedores externos como: 

- Vercel
- Netfly

#### Caso de la vida real

Muy importante usar el server side, en mi caso lo he usado al crear una plataforma donde se requiere mostrar muchos datos. Por ejemplo el codigo de barras para una empresa fabricante de Jeans son muchísimos códigos que maneja. En mi caso cree una vista en la base de datos para unir las consultas y así mostrarla al administrador del sistema.

Use php y javascript para hacer el renderizado desde el lado del cliente .